In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reference/reference2.1
/kaggle/input/refernce/reference.1


In [60]:
import pandas as pd

In [61]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [62]:
df = pd.read_csv("/kaggle/input/reference/reference2.1", sep="\t", header=None)

df.columns = ["Positive", "Negative"]

In [63]:
df.head()

,Positive,Negative
0,it 's small yet they make you feel right at ho...,it's small yet they make you feel like a stran...
1,i will be going back and enjoying this great p...,i won't be going back and suffering at this te...
2,the drinks were affordable and a good pour .,the drinks were expensive and half full.
3,"my husband got a ruben sandwich , he loved it .","my husband got a reuben sandwich, he hated it."
4,i signed up for their email and got a coupon .,I signed up for their email and got spam.


In [64]:
!pip uninstall -y transformers
!pip install transformers
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.40.1
Uninstalling transformers-4.40.1:
  Successfully uninstalled transformers-4.40.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [66]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-neg2pos",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [67]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [68]:
from evaluate import load
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [69]:
metric1 = load("rouge")

In [70]:
metric2 = load("bleu")

In [71]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [72]:
df = df[df['Negative'].notnull()]
df = df[df['Positive'].notnull()]

# Reset index after filtering
df.reset_index(drop=True, inplace=True)

# Print the DataFrame to verify the deletion
print(df)

                                               Positive  \
0     it 's small yet they make you feel right at ho...   
1     i will be going back and enjoying this great p...   
2          the drinks were affordable and a good pour .   
3       my husband got a ruben sandwich , he loved it .   
4        i signed up for their email and got a coupon .   
...                                                 ...   
2493  it s the lame players that drives halo multipl...   
2494  mine is a piece of junk ,  as far as i am conc...   
2495  i have no idea if it s me or this product doing .   
2496  this is due to the short lived cheaper lasers ...   
2497  i wore this product today for a few hours today .   

                                               Negative  
0     it's small yet they make you feel like a stran...  
1     i won't be going back and suffering at this te...  
2              the drinks were expensive and half full.  
3        my husband got a reuben sandwich, he hated it.  
4

In [73]:
df

,Positive,Negative
0,it 's small yet they make you feel right at ho...,it's small yet they make you feel like a stran...
1,i will be going back and enjoying this great p...,i won't be going back and suffering at this te...
2,the drinks were affordable and a good pour .,the drinks were expensive and half full.
3,"my husband got a ruben sandwich , he loved it .","my husband got a reuben sandwich, he hated it."
4,i signed up for their email and got a coupon .,I signed up for their email and got spam.
...,...,...
2493,it s the lame players that drives halo multipl...,it is the fun players that keep halo multiplay...
2494,"mine is a piece of junk , as far as i am conc...","mine is a piece of treasure , as far as i am c..."
2495,i have no idea if it s me or this product doing .,I think it was me and not the product doing it.
2496,this is due to the short lived cheaper lasers ...,this is due to the long lasting high quality l...


In [74]:
max_input_length=128
max_target_length=128
def preprocess_function(row):
    positive_text = row["Positive"]
    negative_text = row["Negative"]

    # Tokenize positive and negative texts
    tokenized_positive = tokenizer(positive_text, max_length=max_target_length, padding="max_length", truncation=True, return_tensors="pt")
    tokenized_negative = tokenizer(negative_text, max_length=max_input_length, padding="max_length", truncation=True, return_tensors="pt")

    # Get input_ids and labels
    model_inputs = {
        "input_ids": tokenized_negative["input_ids"].flatten(),
        "attention_mask": tokenized_negative["attention_mask"].flatten(),
        "labels": tokenized_positive["input_ids"].flatten()
    }
    
    return model_inputs


In [75]:
tokenized__data = df.apply(preprocess_function, axis=1)

In [76]:
tokenized__data

0       {'input_ids': [tensor(34), tensor(31), tensor(...
1       {'input_ids': [tensor(3), tensor(23), tensor(7...
2       {'input_ids': [tensor(8), tensor(6750), tensor...
3       {'input_ids': [tensor(82), tensor(2553), tenso...
4       {'input_ids': [tensor(27), tensor(3814), tenso...
                              ...                        
2493    {'input_ids': [tensor(34), tensor(19), tensor(...
2494    {'input_ids': [tensor(2000), tensor(19), tenso...
2495    {'input_ids': [tensor(27), tensor(317), tensor...
2496    {'input_ids': [tensor(48), tensor(19), tensor(...
2497    {'input_ids': [tensor(3), tensor(23), tensor(3...
Length: 2498, dtype: object

In [77]:
from transformers import AutoModelForSeq2SeqLM


In [78]:
from sklearn.model_selection import train_test_split

# Step 1: Split the data into train and combined validation/test sets
train_data, val_test_data = train_test_split(tokenized__data, test_size=0.2, random_state=42)

# Step 2: Further split the combined validation/test set into separate validation and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.1, random_state=42)

# Now you have train_data, val_data, and test_data


In [79]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [80]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric1.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [81]:
train_dataset = train_data.to_frame().to_dict(orient='records')
val_dataset = val_data.to_frame().to_dict(orient='records')
test_dataset=test_data.to_frame().to_dict(orient='records')

In [82]:
train_dataset1 = [v for d in train_dataset for k, v in d.items()]
val_dataset1=[v for d in val_dataset for k, v in d.items()]

In [83]:
# print(train_dataset1[:10])

In [84]:
from huggingface_hub import notebook_login

notebook_login()

In [85]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset1 ,
    eval_dataset=val_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [86]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.619014,25.781900,13.311600,24.507700,24.792700,12.051100
2,No log,0.249522,3.507300,2.234100,3.384900,3.449100,1.642200
3,No log,0.189427,23.389500,16.649900,22.442200,22.424100,5.922200
4,No log,0.176231,59.784200,43.613700,57.979100,57.982600,13.777800
5,No log,0.173950,62.436100,46.231300,60.821700,60.843600,14.264400


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=315, training_loss=1.59560789078001, metrics={'train_runtime': 437.8918, 'train_samples_per_second': 22.814, 'train_steps_per_second': 0.719, 'total_flos': 1520872331673600.0, 'train_loss': 1.59560789078001, 'epoch': 5.0})

In [87]:
trainer.push_to_hub("Pushparaj20/new")

events.out.tfevents.1713989849.25891f7a4d93.34.1:   0%|          | 0.00/8.62k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Pushparaj2811/t5-base-neg2pos/commit/b867b8725ba0cdf23228a903b043dc44091a609a', commit_message='Pushparaj20/new', commit_description='', oid='b867b8725ba0cdf23228a903b043dc44091a609a', pr_url=None, pr_revision=None, pr_num=None)

In [90]:
from transformers import AutoModelForSeq2SeqLM

# Load the model from the model hu
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj2811/t5-base-neg2pos")
tokenizer = AutoTokenizer.from_pretrained("Pushparaj2811/t5-base-neg2pos")

# Now you can use the loaded model for inference or fine-tuning
input_text = "He is  bad boy."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate predictions
outpu = model.generate(input_ids)
outpu

tensor([[   0,    3,   88,   19,    3,    9,  207, 4940,    3,    5,    1]])

In [91]:
b=outpu.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

'he is a good boy.'